# Notebook Setup

In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from IPython.display import display, Math


import numpy as np
import matplotlib.pyplot as plt

import os
import sys

sys.path.insert(1, '../')

from moments_toolkit import moments_toolkit

from moments_toolkit import ratio_formula, sum_ratios_formula

import gvar as gv

#from utilities import plateau_search_symm

import itertools as it

import correlatoranalyser as CA

from moments_toolkit import average_moments_over_T
from utilities import jackknife_resamples, bootstrap_resamples

In [2]:
p3fold = os.environ['mount_point_path'] + "48c48/binned_1012_hmz370_BMW_extended/3PointCorrelation/"
p2fold = os.environ['mount_point_path'] + "48c48/binned_1012_hmz370_BMW_extended/2PointCorrelation/"


opAnalyzer = moments_toolkit(p3fold, p2fold,
                            skip3p=False, skipop=False,
                            verbose=True,
                            fast_data_folder = "../fast_data_extended_p0_q0",
                            operator_folder= "../operator_database",
                            momentum='PX0_PY0_PZ0',
                            insertion_momentum = 'qx0_qy0_qz0',
                            tag_2p='hspectrum',
                            max_n=2 #max_n=3
                            )


Initializing the moments_toolkit class instance...


Initializing the building block class instance...


Reading the the keys of the dataset ...


Reading the 2 point and 3 point correlators from the fast access dataset ...


Reading the list of all the available operators from the database...


Class Initialization Complete!



In [3]:
opAnalyzer.set_resampling_type('jackknife')
opAnalyzer.set_fit_parms(central_value_fit=True, central_value_fit_correlated=True, resample_fit=True, resample_fit_correlated=True)

In [4]:
#we take the operators of the paper

opV1 = 1/6 * opAnalyzer.get_operator(2)
opV2 = 1/(3 * np.sqrt(2)) * (opAnalyzer.get_operator(2) - opAnalyzer.get_operator(3))
opV3 = 1/np.sqrt(2) * opAnalyzer.get_operator(14)

opA1 = 1/np.sqrt(2) * opAnalyzer.get_operator(28)
opA2 = 1/np.sqrt(2) * opAnalyzer.get_operator(32)

opT1 = opAnalyzer.get_operator(74) + 1/2 * opAnalyzer.get_operator(78)
opT2 = opAnalyzer.get_operator(78)
opT3 =  1/6 * ( -3 * opAnalyzer.get_operator(83) + 2 * opAnalyzer.get_operator(87) + 3* opAnalyzer.get_operator(91) + opAnalyzer.get_operator(95) )
opT4 = 1/2 * ( opAnalyzer.get_operator(83) + 2 * opAnalyzer.get_operator(87) -2* opAnalyzer.get_operator(91) )



#we empty the selected op just to be sure
opAnalyzer.deselect_operator()

#we append the operators
opAnalyzer.append_operator(opV1)
opAnalyzer.append_operator(opV2)
opAnalyzer.append_operator(opV3)
opAnalyzer.append_operator(opA1)
opAnalyzer.append_operator(opA2)
opAnalyzer.append_operator(opT1)
opAnalyzer.append_operator(opT2)
opAnalyzer.append_operator(opT3)
opAnalyzer.append_operator(opT4)

#opAnalyzer.append_operator(opAnalyzer.get_operator(125))


#we deselect the op with 0 kin factor
Klist = opAnalyzer.get_Klist()
eliminate_op = [op for op,kin in zip(opAnalyzer.selected_op,Klist) if np.abs( kin.mean ) == 0]
for op in eliminate_op:
    opAnalyzer.deselect_operator(op)
Klist = opAnalyzer.get_Klist()


opAnalyzer.display_operators()

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

# One Operator Analysis

In [324]:
iop = 0

## Plateau Search

In [325]:
from utilities import redchi2_cov

#function that given a 1D array returns the values of the indices identifying its plateau (the first and last index), that is symmetric around its middle point
def plateau_search_symm(in_array: np.ndarray, covmat: np.ndarray, chi2_treshold:float=1.0, only_sig:bool=True) -> tuple[int,int]:
    """
    Input:
        - in_array: the 1D array we want to search the plateau of
        - covmat: a 2D array, representing the covariance matrix of in_array
        - chi2_treshold: the treshold for the plateau determination
        - only_sig: bool, if True only the standard deviation, and not the whole cavariance matrix, is used for the plateau determination
    
    Output:
        - (start_plateau,end_plateau): indices such that in_array[start_plateau,end_plateau] is the region with the plateau
    """

    #first we compute the len of the array
    len_array = np.shape(in_array)[0]

    #we loop over all the possible plateau lenghts, starting from the biggest possible one and then diminishing it up to a plataeau of len 2
    for len_plat in range(len_array,1,-2):

        start_plateau = int( (len_array-len_plat)/2 )

        #the suggested plateau region in this case is
        plat = in_array[start_plateau:start_plateau+len_plat]

        #we also have to reshape the covariance matrix
        covmat_plat = covmat[start_plateau:start_plateau+len_plat, start_plateau:start_plateau+len_plat]

        #the value of the plateau is
        plat_value = np.average(plat, weights = np.diag(np.linalg.inv(covmat_plat)), axis=0, keepdims=True) #the weights are the inverse of the sigma squared

        #we compute the chi2 of the current plateau
        chi2 = redchi2_cov(plat, plat_value, covmat_plat,only_sig=only_sig)

        #we see if the chi2 meets the condition
        if chi2 < chi2_treshold: #TO DO: in this case put the value in a list and then at the end of the inner loop search for the better one

            return start_plateau, start_plateau+len_plat
                
    #if by the end of the loop the chi2 condition is never met (i.e. if len_plat is 1) we return the point corresponding to the middle of the dataset
    #return int(len_array/2), int(len_array/2)+2-len_array%2
    return None

In [326]:
opAnalyzer.remove_T(3,12,verbose=True) #3 too few points, 12 too many
Rmean,Rstd,Rcov = opAnalyzer.get_R()

"(Nop,NT,maxT+1)"
Rmean.shape

Tlist = opAnalyzer.chosen_T_list


Available source-skink separation values: [3, 4, 5, 6, 7, 8, 10, 12]
Chosen source-sink separation values: [4, 5, 6, 7, 8, 10]


'(Nop,NT,maxT+1)'

(5, 6, 11)

In [327]:
cut_dict = {}
N_points_dict = {}

for iT,T in enumerate(Tlist):
    T, plateau_search_symm(Rmean[iop,iT,:T+1],Rcov[iop,iT,:T+1,:T+1],only_sig=True, chi2_treshold=5.0)
    cut = plateau_search_symm(Rmean[iop,iT,:T+1],Rcov[iop,iT,:T+1,:T+1],only_sig=True, chi2_treshold=5.0)
    cut_dict[T] = cut
    N_points_dict[T] = cut[1]-cut[0] if cut is not None else 0

(4, None)

(5, (2, 4))

(6, (2, 5))

(7, (3, 5))

(8, (3, 6))

(10, (2, 9))

In [328]:
cut_dict
N_points_dict

{4: None, 5: (2, 4), 6: (2, 5), 7: (3, 5), 8: (3, 6), 10: (2, 9)}

{4: 0, 5: 2, 6: 3, 7: 2, 8: 3, 10: 7}

## Data Preparation

In [329]:
abscissa = np.array([
    [T, tau] for T in Tlist if cut_dict[T] is not None for tau in np.arange(0,T+1)[cut_dict[T][0]:cut_dict[T][1]] 
])

In [330]:
cut_dict[6]

(2, 5)

In [331]:
abscissa

array([[ 5,  2],
       [ 5,  3],
       [ 6,  2],
       [ 6,  3],
       [ 6,  4],
       [ 7,  3],
       [ 7,  4],
       [ 8,  3],
       [ 8,  4],
       [ 8,  5],
       [10,  2],
       [10,  3],
       [10,  4],
       [10,  5],
       [10,  6],
       [10,  7],
       [10,  8]])

In [332]:
#np.linspace(cut_dict[6][0],cut_dict[6][1]-1,20)

In [333]:
# abscissa = np.array([
#     [t, tau] for t in Tlist for tau in np.arange(1,t)[2:-2]
# ])
# abscissa

In [334]:
#We first take the 3 point and 2 point correlators needed to compute the ratio
p3_corr = opAnalyzer.get_p3corr() #shape = (nop, nconf, nT, maxT+1)
p2_corr = opAnalyzer.get_p2corr() #shape = (nconf, latticeT)


Nbst = 100
Ratios_resamples = {}

#we loop over all the T values we have
for iT,T in enumerate(opAnalyzer.chosen_T_list):

    #we perform the jackknife analysis (the observable being the ratio we want to compute)
    Ratios_resamples[T] = bootstrap_resamples([p3_corr[iop,:,iT,:], p2_corr], lambda x,y: ratio_formula(x,y, T=T, gauge_axis=0), res_axis_list=[0,0], Nres=Nbst, new_resamples=False)[:,:T+1]
    #Ratios_bst[T] = jackknife_resamples([p3_corr[0,:,iT,:], p2_corr], lambda x,y: ratio_formula(x,y, T=T, gauge_axis=0), res_axis_list=[0,0])[:,:T+1]


Ratio_ror = np.zeros( (Nbst, len(abscissa)) )

for idx, (t,tau) in enumerate(abscissa):

    Ratio_ror[:,idx] = Ratios_resamples[t][:,tau]

In [335]:
Ratio_ror

array([[0.07670187, 0.07638937, 0.07346173, ..., 0.05871958, 0.06372244,
        0.07298985],
       [0.07633157, 0.07628737, 0.07261673, ..., 0.06242318, 0.06927269,
        0.07804925],
       [0.07757457, 0.07707951, 0.07424805, ..., 0.0612452 , 0.06476651,
        0.07214532],
       ...,
       [0.0761092 , 0.07584626, 0.07219474, ..., 0.06207655, 0.06772353,
        0.07204645],
       [0.07649312, 0.07604507, 0.072775  , ..., 0.06263679, 0.07091053,
        0.07482713],
       [0.07728771, 0.07651833, 0.0738974 , ..., 0.06358255, 0.06916406,
        0.0742171 ]], shape=(100, 17))

## Model

In [336]:
class SymmetricRatioModel:
    def __init__(self, number_states_sink, number_states_source, include_mix_term = True):
        self.number_states_sink = number_states_sink
        self.number_states_source = number_states_source
        self.include_mix_term = include_mix_term
        
        # GS
        if   number_states_sink == 1 and number_states_source == 1:
            self.Nparams = 1
        # GS + ES @ sink 
        elif number_states_sink == 2 and number_states_source == 1:
            self.Nparams = 3
        # GS + ES @ source
        elif number_states_sink == 1 and number_states_source == 2:
            self.Nparams = 3
        # GS + ES @ source + ES @ sink + ES @ (source & sink)
        elif number_states_sink == 2 and number_states_source == 2 and include_mix_term:
            self.Nparams = 4
        # GS + ES @ source + ES @ sink + ES @ (source & sink)
        elif number_states_sink == 2 and number_states_source == 2:
            self.Nparams = 3
        else:
            raise NotImplementedError("Model only implemented for at most one excited state at source and/or sink")

    def __call__(self, t_tau, p):
        r"""
            parameter:
                - E-m = E_N(q)-m_N > 0 (Ground state exponent)
                - dE{n}(q) = E_n(q) - E_N(q) > 0 (relative energy/mass if q=0)
                    - Currently accepted: dE1(q), dE1(0)
                - Amn (replace m,n by integers, matrix element for the mth excited state at source, and nth excited state a sink)
                    - A00 is the ground state matrix element
        """
        t = t_tau[:,0]
        tau = t_tau[:,1]

        # The ground state contribution (exponential is factorized)
        out = np.full_like(tau, p['A00'], dtype = object)

        # The excited state at source 
        if self.number_states_source == 2:
            out += p["A01"] * np.exp(                        - tau * p["dE1(0)"])

        # The excited state at sink
        if self.number_states_sink == 2:
            out += p["A01"] * np.exp( -(t-tau) * p["dE1(0)"]                    ) 

        # The excited states at source and sink 
        if self.number_states_source == 2 and self.number_states_sink == 2 and self.include_mix_term:
            out += p["A11"] * np.exp( -(t-tau) * p["dE1(0)"] - tau * p["dE1(0)"])  

        return out

    def flat_prior(self,sign,dE_mean):
        prior = gv.BufferDict()

        prior["A00"] = sign*gv.gvar(1,100) 
        
        # The excited state at source 
        if self.number_states_source == 2:
            prior["log(dE1(0))"] = gv.log(gv.gvar(dE_mean,10*dE_mean)) #gv.log(gv.gvar(1, 100))
            prior["A01"]    = sign*gv.gvar(1,100) 

        # The excited state at sink
        if self.number_states_sink == 2:
            prior["log(dE1(0))"] = gv.log(gv.gvar(dE_mean,10*dE_mean)) #gv.log(gv.gvar(1, 100))
            prior["A01"]    = sign*gv.gvar(1,100) 

        # The excited states at source and sink 
        if self.number_states_source == 2 and self.number_states_sink == 2 and self.include_mix_term:
            prior["A11"]    = sign*gv.gvar(1,100)  
        
        return prior
    
    def guess_prior(self,sign,dE_mean):
        prior = gv.BufferDict()

        prior["A00"] = sign*gv.gvar(1,0.5) 
        
        # The excited state at source 
        if self.number_states_source == 2:
            prior["log(dE1(0))"] = gv.log(gv.gvar(dE_mean,dE_mean)) #gv.log(gv.gvar(0.1, 1))
            prior["A01"]    =sign*gv.gvar(1e-2,1)

        # The excited state at sink
        if self.number_states_sink == 2:
            prior["log(dE1(0))"] = gv.log(gv.gvar(dE_mean,dE_mean)) #gv.log(gv.gvar(0.1, 1))
            prior["A01"]    = sign*gv.gvar(1e-2,1)

        # The excited states at source and sink 
        if self.number_states_source == 2 and self.number_states_sink == 2 and self.include_mix_term:
            prior["A11"]    = sign*gv.gvar(0.1,1)   
        
        return prior
    
    def model_prior(self, dE, matele, abscissa, ratio):
        prior = gv.BufferDict()

        r1_list=[]
        for i_probe in range(len(abscissa)):
            Tprobe,tprobe = abscissa[i_probe]
            Rprobe = ratio[i_probe]
            r1_list.append( (Rprobe - matele.mean  ) * np.exp(Tprobe/2 * dE.mean) / np.cosh((Tprobe/2-tprobe)*dE.mean) )
        r1_mean = np.mean(r1_list)

        prior["A00"] = matele
        
        # The excited state at source 
        if self.number_states_source == 2:
            prior["log(dE1(0))"] = gv.log(dE)
            prior["A01"]    = gv.gvar(r1_mean,10*r1_mean)

        # The excited state at sink
        if self.number_states_sink == 2:
            prior["log(dE1(0))"] = gv.log(dE)
            prior["A01"]    = gv.gvar(r1_mean,10*r1_mean)

        # The excited states at source and sink 
        if self.number_states_source == 2 and self.number_states_sink == 2 and self.include_mix_term:
            prior["A11"]    = gv.gvar(r1_mean,10*r1_mean)   
        
        return prior

In [337]:
model = SymmetricRatioModel(number_states_sink=2,number_states_source=2, include_mix_term = True)

In [338]:




opAnalyzer.remove_T()
matele_fromS = opAnalyzer.get_M_from_S(method="fit", tskip_list=[1,2,3],moments=False)
dE = opAnalyzer.get_dE()
opAnalyzer.remove_T(3,12)
mat_ele_avg, iTmin  = average_moments_over_T(matele_fromS[iop], chi2=10.0)



guess_prior = model.guess_prior(sign=1 if mat_ele_avg.mean>0 else -1,dE_mean=dE.mean)

flat_prior = model.flat_prior(sign=1 if mat_ele_avg.mean>0 else -1,dE_mean=dE.mean)

model_prior = model.model_prior(dE=gv.gvar(dE.mean,dE.mean),matele=mat_ele_avg,abscissa=abscissa, ratio=np.mean(Ratio_ror,axis=0))

## Fit

In [339]:
fit_result = CA.fit(
    abscissa = abscissa,
    ordinate_est = np.mean( Ratio_ror, axis = 0 ),
    ordinate_std = np.std ( Ratio_ror, axis = 0 ),
    ordinate_cov = np.cov ( Ratio_ror, rowvar = False ),
    resample_ordinate_est = Ratio_ror,
    resample_ordinate_std = np.std ( Ratio_ror, axis = 0 ),
    resample_ordinate_cov = np.cov ( Ratio_ror, rowvar = False ),
    
    central_value_fit = True,
    central_value_fit_correlated = True,
    resample_fit = True,
    resample_fit_correlated = True,
    resample_fit_resample_prior = False,
    resample_type = "bst",
    # Fitting infos
    model = model,
    #prior = model_prior,
    #prior=flat_prior,
    prior=guess_prior,
)

In [340]:
fit_result

FitResult[ ([5 2],[10  8]), Ndata=17, resample:bst]:
  𝜒²/dof [dof] = 1.35 [17]
  AIC = 0.348 
    - A00: 0.0540(48)  [1.00(50)]
    - dE1(0): 0.474(74)  [0.082(82)]
    - A01: 0.0362(44)  [0.01(1.00)]
    - A11: 0.0004(227)  [0.1(1.0)]

In [341]:
#opAnalyzer.get_dE(force_fit=True, units="MeV")
#
#opAnalyzer.get_dE(force_fit=True, units="lattice")

In [342]:
fit_result.best_fit_param["A00"] /Klist[iop]

0.193(15)

In [343]:
COLORS = ["red","blue","green", "orange", "purple", "brown", "pink", "black"]

_=plt.figure()

for iT, T in enumerate(Tlist):

    norm_K = Klist[iop].mean
    taus = np.arange(1,T)

    _=plt.errorbar(
        taus - T/2,
        np.mean( Ratios_resamples[T][:,1:-1], axis = 0 )/norm_K,
        np.std ( Ratios_resamples[T][:,1:-1], axis = 0 )/np.abs(norm_K),
        fmt = '.:',
        capsize = 2,
        label = f"${T=}$",
        color = COLORS[iT]
    )

    if cut_dict[T] is not None:

        eps=0.1
        cont_taus = np.linspace(cut_dict[T][0]-eps, cut_dict[T][1]-1+eps,100)

        abscissa = np.array([
            (T, tau) for tau in cont_taus
        ])

        fit_ordinate = fit_result.eval( abscissa )

        _=plt.plot(cont_taus-T/2, fit_ordinate["est"] / norm_K, color = COLORS[iT])
        
        _=plt.fill_between(
            cont_taus-T/2, 
            (fit_ordinate["est"]+fit_ordinate["err"]) / norm_K,
            (fit_ordinate["est"]-fit_ordinate["err"]) / norm_K,
            color = COLORS[iT],
            alpha = 0.2
        )

_=plt.xlabel(r"$\tau - t/2$")
_=plt.ylabel(r"$R(t,\tau)$")
_=plt.show()

In [344]:
num = 8
int(num/2)
int(num/2)+1 + num%2

4

5

In [345]:
#opAnalyzer.get_E(force_fit=True)

## Number of Points Increase

In [346]:
def abscissa_ratio_from_cutdict(cutdict):
    abscissa = np.array([
        [T, tau] for T in Tlist if cutdict[T] is not None for tau in np.arange(0,T+1)[cutdict[T][0]:cutdict[T][1]] 
    ])

    Ratio_ror = np.zeros( (Nbst, len(abscissa)) )

    for idx, (t,tau) in enumerate(abscissa):

        Ratio_ror[:,idx] = Ratios_resamples[t][:,tau]

    return abscissa, Ratio_ror

In [347]:
from copy import deepcopy

In [348]:
#cut_dict_old = { k:v for k,v in cut_dict.items()}
#N_points_dict_old = { k:v for k,v in N_points_dict.items()}

cut_dict_old = deepcopy(cut_dict)
N_points_dict_old = deepcopy(N_points_dict)

In [349]:
#we instanatiate the states of the fits we want to do
fit_state = CA.FitState()

for mix_term in [False,True]:

    model = SymmetricRatioModel(number_states_sink=2,number_states_source=2, include_mix_term = mix_term)

    flat_prior = model.flat_prior(sign=1 if mat_ele_avg.mean>0 else -1,dE_mean=dE.mean)

    abscissa, Ratio_ror = abscissa_ratio_from_cutdict(cut_dict)

    #first fit 
    #............
    fit_result = CA.fit(
                        abscissa = abscissa,
                        ordinate_est = np.mean( Ratio_ror, axis = 0 ),
                        ordinate_std = np.std ( Ratio_ror, axis = 0 ),
                        ordinate_cov = np.cov ( Ratio_ror, rowvar = False ),
                        resample_ordinate_est = Ratio_ror,
                        resample_ordinate_std = np.std ( Ratio_ror, axis = 0 ),
                        resample_ordinate_cov = np.cov ( Ratio_ror, rowvar = False ),
                        
                        central_value_fit = True,
                        central_value_fit_correlated = True,
                        resample_fit = True,
                        resample_fit_correlated = True,
                        resample_fit_resample_prior = False,
                        resample_type = "bst",
                        # Fitting infos
                        model = model,
                        #prior = model_prior,
                        prior=flat_prior,
                        #prior=guess_prior,
                        )
    fit_state.append(fit_result)

    cut_dict = deepcopy(cut_dict_old)
    N_points_dict = deepcopy(N_points_dict_old)
    print(cut_dict_old)

    while True:
        
        fit_done=False

        currentT_list = list(cut_dict.keys())
        cut_list = list(cut_dict.values())
        Npoints_list = list(N_points_dict.values())

        for i, (T, cuts, Npoints) in enumerate(zip(currentT_list,cut_list,Npoints_list)):
            #i,T,cuts,Npoints

            if i==0: continue

            if Npoints < list(N_points_dict.values())[i-1] and Npoints <T:
                
                if Npoints!=0:
                    cut_dict[T] = (cut_dict[T][0]-1,cut_dict[T][1]+1)
                    N_points_dict[T] += 2
                else:
                    cut_dict[T] = (int(T/2), int(T/2)+1 + T%2)
                    N_points_dict[T] += 1 + T%2

                #do other fit.......
                abscissa, Ratio_ror = abscissa_ratio_from_cutdict(cut_dict)

                fit_result = CA.fit(
                                    abscissa = abscissa,
                                    ordinate_est = np.mean( Ratio_ror, axis = 0 ),
                                    ordinate_std = np.std ( Ratio_ror, axis = 0 ),
                                    ordinate_cov = np.cov ( Ratio_ror, rowvar = False ),
                                    resample_ordinate_est = Ratio_ror,
                                    resample_ordinate_std = np.std ( Ratio_ror, axis = 0 ),
                                    resample_ordinate_cov = np.cov ( Ratio_ror, rowvar = False ),
                                    
                                    central_value_fit = True,
                                    central_value_fit_correlated = True,
                                    resample_fit = True,
                                    resample_fit_correlated = True,
                                    resample_fit_resample_prior = False,
                                    resample_type = "bst",
                                    # Fitting infos
                                    model = model,
                                    #prior = model_prior,
                                    prior=flat_prior,
                                    #prior=guess_prior,
                                    )
                fit_state.append(fit_result)
                fit_done = True

        if fit_done==False: break

{4: None, 5: (2, 4), 6: (2, 5), 7: (3, 5), 8: (3, 6), 10: (2, 9)}
{4: None, 5: (2, 4), 6: (2, 5), 7: (3, 5), 8: (3, 6), 10: (2, 9)}


In [350]:
cut_dict

{4: None, 5: (2, 4), 6: (2, 5), 7: (2, 6), 8: (2, 7), 10: (2, 9)}

In [351]:
fit_state

FitState(fit_results=[FitResult[ ([5 2],[10  8]), Ndata=21, resample:bst]:
  𝜒²/dof [dof] = 0.964 [21]
  AIC = -14.3 
    - A00: 0.0548(18)  [1(100)]
    - dE1(0): 0.468(28)  [0.08(82)]
    - A01: 0.0344(15)  [1(100)]
, FitResult[ ([5 2],[10  8]), Ndata=19, resample:bst]:
  𝜒²/dof [dof] = 0.919 [19]
  AIC = -12.9 
    - A00: 0.0554(19)  [1(100)]
    - dE1(0): 0.474(29)  [0.08(82)]
    - A01: 0.0341(16)  [1(100)]
, FitResult[ ([5 2],[10  8]), Ndata=21, resample:bst]:
  𝜒²/dof [dof] = 0.943 [21]
  AIC = -11.7 
    - A00: 0.0567(39)  [1(100)]
    - dE1(0): 0.518(79)  [0.08(82)]
    - A01: 0.0332(32)  [1(100)]
    - A11: 0.018(27)  [1(100)]
, FitResult[ ([5 2],[10  8]), Ndata=21, resample:bst]:
  𝜒²/dof [dof] = 0.943 [21]
  AIC = -11.7 
    - A00: 0.0567(39)  [1(100)]
    - dE1(0): 0.518(79)  [0.08(82)]
    - A01: 0.0332(32)  [1(100)]
    - A11: 0.018(27)  [1(100)]
, FitResult[ ([5 2],[10  8]), Ndata=19, resample:bst]:
  𝜒²/dof [dof] = 0.913 [19]
  AIC = -9.79 
    - A00: 0.0560(43)  [1(10

In [352]:
def weights_from_fitstate(fitstate):
    AIC_array =np.array( [fitres.AIC for fitres in fitstate] )
    AIC_min = np.min(AIC_array)
    weights = np.exp(-0.5 * (AIC_array - AIC_min)) 
    return weights/np.sum(weights)

In [353]:
result_dict = {parm:gv.gvar(fit_state.model_average()["est"][parm], fit_state.model_average()["err"][parm]) for parm in fit_state.model_average()["est"] }

result_dict

result_dict["A00"] / Klist[iop]

opAnalyzer.lattice_to_MeV(result_dict['dE1(0)'])

{'A00': 0.0554(25), 'dE1(0)': 0.484(49), 'A01': 0.0341(21), 'A11': 0.017(26)}

0.1949(88)

821(83)

In [354]:
weights_from_fitstate(fit_state)

array([0.44622542, 0.22225108, 0.11979921, 0.11979921, 0.045977  ,
       0.04594807])

In [355]:
COLORS = ["red","blue","green", "orange", "purple", "brown", "pink", "black"]

_=plt.figure()

for iT, T in enumerate(Tlist):

    norm_K = Klist[iop].mean
    taus = np.arange(1,T)

    _=plt.errorbar(
        taus - T/2,
        np.mean( Ratios_resamples[T][:,1:-1], axis = 0 )/norm_K,
        np.std ( Ratios_resamples[T][:,1:-1], axis = 0 )/np.abs(norm_K),
        fmt = '.:',
        capsize = 2,
        label = f"${T=}$",
        color = COLORS[iT]
    )

    if cut_dict[T] is not None:

        eps=0.1
        cont_taus = np.linspace(cut_dict[T][0]-eps, cut_dict[T][1]-1+eps,100)

        abscissa = np.array([
            (T, tau) for tau in cont_taus
        ])

        fit_ordinate_array = np.array( [ gv.gvar( fitresult.eval( abscissa )["est"], fitresult.eval( abscissa )["err"] ) for fitresult in fit_state] )
        fit_ordinate = np.average(fit_ordinate_array, axis=0, weights=weights_from_fitstate(fit_state))

        ordinate_mean = np.array( [ordinate.mean for ordinate in fit_ordinate] )
        ordinate_high = np.array( [ordinate.mean + ordinate.sdev for ordinate in fit_ordinate] )
        ordinate_low = np.array( [ordinate.mean - ordinate.sdev for ordinate in fit_ordinate] )

        _=plt.plot(cont_taus-T/2, ordinate_mean / norm_K, color = COLORS[iT])
        
        _=plt.fill_between(
            cont_taus-T/2, 
            ordinate_high / norm_K,
            ordinate_low / norm_K,
            color = COLORS[iT],
            alpha = 0.2
        )

_=plt.xlabel(r"$\tau - t/2$")
_=plt.ylabel(r"$R(t,\tau)$")
_=plt.show()